In [1]:
!python3 --version

Python 3.10.12


In [18]:
!pip install openai==1.10.0
!pip install langchain==0.1.3
!pip install langchain-openai==0.0.3
!pip install -U langchain_openai==0.0.5
!pip install langchain-community==0.0.15
!pip install langchain-core==0.1.15
!pip install langsmith==0.0.83
!pip install langgraph==0.0.21
!pip install numexpr==2.9.0
!pip install wikipedia
!pip install -U duckduckgo-search==4.4

  Using cached langchain_openai-0.0.3-py3-none-any.whl.metadata (2.5 kB)
Using cached langchain_openai-0.0.3-py3-none-any.whl (28 kB)
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.0.5
    Uninstalling langchain-openai-0.0.5:
      Successfully uninstalled langchain-openai-0.0.5
  Using cached langchain_openai-0.0.5-py3-none-any.whl.metadata (2.5 kB)
Using cached langchain_openai-0.0.5-py3-none-any.whl (29 kB)
  Attempting uninstall: langchain_openai
    Found existing installation: langchain-openai 0.0.3
    Uninstalling langchain-openai-0.0.3:
      Successfully uninstalled langchain-openai-0.0.3
  Using cached langchain_core-0.1.15-py3-none-any.whl.metadata (6.0 kB)
Using cached langchain_core-0.1.15-py3-none-any.whl (229 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.18
    Uninstalling langchain-core-0.1.18:
      Successfully uninstalled langchain-core-0.1.18
ERROR: pip's dependency reso

In [19]:
import os
import random
import time

from typing import Any, List, Optional, Sequence, Tuple, Union

In [20]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [21]:
def load_dotenv(dotenv_path=".env"):
    with open(dotenv_path) as f:
        for line in f:
            if line.startswith('#') or not line.strip():
                continue
            # 環境変数を設定
            key, value = line.strip().split('=', 1)
            os.environ[key] = value

# .envファイルを読み込む
load_dotenv()


In [22]:
# 環境変数を使用する
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

In [23]:
import os


# def _set_if_undefined(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass(f"Please provide your {var}")


# # 必須APIキーの確認
# _set_if_undefined("OPENAI_API_KEY")
# _set_if_undefined("LANGCHAIN_API_KEY")
# _set_if_undefined("TAVILY_API_KEY")

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
# LangSmithの設定
# os.environ["LANGCHAIN_TRACING_V2"]="true"
# os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "blog_supervisor_dev"

# LLM_SMART_MODEL = "gpt-3.5-turbo-1106"
# LLM_SMART_MODEL = "gpt-3.5-turbo-0125"
# LLM_SMART_MODEL = "gpt-4-1106-preview"
LLM_SMART_MODEL = "gpt-4-0125-preview"

## エージェントを生成するユーティリティ関数の定義
エージェントの実装が面倒なので、LangChainのAgentExecutor

In [24]:
from langchain_core.runnables import Runnable
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from typing import TypedDict

In [25]:
class AgentDescription(TypedDict):
    name: str
    description: str
    
def create_agent(
        llm: ChatOpenAI,
        tools: list,
        system_prompt: str,
) -> AgentExecutor:
    system_prompt += "\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason! You are one of the following team members: {team_members}."
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools)


def create_team_supervisor(
        llm: ChatOpenAI,
        system_prompt: str,
        members: list[AgentDescription]
) -> Runnable:
    member_names = [member["name"] for member in members]
    team_members = []
    for member in members:
        team_members.append(f"name: {member['name']}\ndescription: {member['description']}")
    options = ["FINISH"] + member_names
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of option: {options}",
            ),
        ]
    ).partial(options=str(options), team_members="\n\n".join(team_members))
    return (
            prompt
            | llm.bind_functions(functions=[function_def], function_call="route")
            | JsonOutputFunctionsParser()
    )

## Researchエージェントの定義
Web検索にTavilyを利用している。 TavilyもしくはURLの直接アクセスをサポートするために、 function callingを利用したエージェントを利用

In [26]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults, DuckDuckGoSearchRun

llm = ChatOpenAI(model_name=LLM_SMART_MODEL, 
                 temperature=0.7)
# tavily_tool = TavilySearchResults(max_results=5)
tavily_tool = DuckDuckGoSearchRun(max_results=5)

@tool
def scrape_webpages(urls: list[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )


def create_research_agent() -> Runnable:
    prompt = "You are a research assistant who can search for up-to-date info using the tavily search engine."
    return create_agent(llm, [tavily_tool, scrape_webpages], prompt)


## LangGraphに設定するノードの定義
ノードには関数またはRunnableが設定できる。 今回はステートの更新差分を分かりやすくするために関数を設定している。

In [27]:
import operator
from typing import Annotated
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import BaseMessage, AIMessage

In [28]:
# 各ノードの名称を定義
RESEARCH_NODE = "research"
QUALITY_ASSURANCE_NODE = "quality_assurance"
WRITER_NODE = "writer"
SUPERVISOR_NODE = "supervisor"

# チームメンバーの定義
team_members = [
    {"name": RESEARCH_NODE,
     "description": "Search the web for necessary information and write articles as requested by users."},
    {"name": QUALITY_ASSURANCE_NODE,
     "description": f"Check that the quality of the article meets the criteria. If not, return feedback. The acceptability of deliverables is always determined by {QUALITY_ASSURANCE_NODE}."},
    {"name": WRITER_NODE,
     "description": "Write articles that even the world's best AI checkers would judge as written by a human."},
]


# トップレベルのステートを定義
class TopState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]
    output: str
    next: str


def create_llm_chain(llm: ChatOpenAI, system_message: str) -> Runnable:
    prompt = ChatPromptTemplate.from_messages(
        [("system", system_message), ("user", "{message}")]
    )
    chain = {"message": RunnablePassthrough()} | prompt | llm | StrOutputParser()
    return chain


# Researcherノードの定義
def research_node(state: TopState) -> dict:
    last_message = state["messages"][-1]
    result = create_research_agent().invoke({"messages": [last_message]})
    return {
        "output": result["output"],
        "messages": [AIMessage(content=result["output"])]
    }


# Writerノードの定義
def writer_node(state: TopState) -> dict:
    system_message = """You are a writing specialist.
You can write sentences that even the world's best AI checkers would judge as written by a human."""
    latest_message = state["messages"][-1].content
    chain = create_llm_chain(llm, system_message)
    result = chain.invoke(latest_message)
    return {
        "output": result,
        "messages": [AIMessage(content=result)]
    }


# 品質チェックノードの定義
def qa_node(state: TopState) -> dict:
    system_message = """You are a specialist in inspecting the quality of articles. 
Inspect the article to see if it meets the following requirements:

- The article is written in Japanese.
- The writing style MUST be such that the AI sentence checker determines that it was written by a HUMAN.
- The article MUST be written in a way that is easy to understand.
- The article MUST meet the user's requirements.

If the evaluation criteria are met, write "Acceptable".
In addition, write the reason why you judged that the evaluation criteria are met.

If the evaluation criteria are not met, write "Not Acceptable".
In addition, provide feedback on what needs to be done to meet the evaluation criteria.

DO NOT make excuses such as "I can't make a decision because I am an AI".

The quality of your articles is relevant to your career.
Please be as rigorous as possible in your inspections and make sure that your feedback is helpful in making corrections.
"""
    latest_output = state["output"]
    chain = create_llm_chain(llm, system_message)
    result = chain.invoke(latest_output)
    result_with_original_article = f"feedback: {result}\n\n-----\n{latest_output}"
    return {"messages": [AIMessage(content=result_with_original_article)]}


# Supervisorノードの定義
def supervisor_node(state: TopState) -> Runnable:
    prompt = """You are a supervisor tasked with managing a conversation between the following teams:
{team_members}
    
Given the following user request, respond with the worker to act next. 
Each worker will perform a task and respond with their results and status.
When finished, respond with FINISH."""
    return create_team_supervisor(llm, prompt, team_members)

## LangGraphの定義
LangGraphのStateGraphにノードと、ノード間を繋ぐエッジを設定する。 SUPERVISOR_NODEは次にどのノードに遷移するかを決定するため、条件付きエッジを設定する。

In [29]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, END

graph = StateGraph(TopState)

graph.add_node(RESEARCH_NODE, research_node)
graph.add_node(QUALITY_ASSURANCE_NODE, qa_node)
graph.add_node(WRITER_NODE, writer_node)
graph.add_node(SUPERVISOR_NODE, supervisor_node)

graph.add_edge(RESEARCH_NODE, SUPERVISOR_NODE)
graph.add_edge(QUALITY_ASSURANCE_NODE, SUPERVISOR_NODE)
graph.add_edge(WRITER_NODE, SUPERVISOR_NODE)
graph.add_conditional_edges(
    SUPERVISOR_NODE,
    lambda x: x["next"],
    {
        RESEARCH_NODE: RESEARCH_NODE,
        QUALITY_ASSURANCE_NODE: QUALITY_ASSURANCE_NODE,
        WRITER_NODE: WRITER_NODE,
        "FINISH": END,
    }
)

graph.set_entry_point(SUPERVISOR_NODE)
blog_writer = {"messages": lambda x: [HumanMessage(content=x)]} | graph.compile()


## LangGraphの実行
処理の流れが分かりやすいようにstream関数で実行している。 LangSmithが利用できる場合はLangSmith上で確認すると、より分かりやすい。

In [30]:
query = """
以下の論文サイトについて、概要、新規性、数式やアルゴリズムなどの手法、実験結果、実装方法等を順番に考えながら詳しく日本語で書きなさい。
https://osu-nlp-group.github.io/TravelPlanner/
"""
with Timer(prefix=f'Search agent operating time: '):
    latest_output = ""
    cnt = 0
    for s in blog_writer.stream(query, {"recursion_limit": 100}):
        if cnt == 4:
            break
        if "__end__" not in s:
            print(s)
            print("---")
            writing_output = (
                    s.get(RESEARCH_NODE, {}).get("output") or
                    s.get(WRITER_NODE, {}).get("output")
            )
            if writing_output:
                latest_output = writing_output
        cnt += 1
        print(cnt)



{'supervisor': {'next': 'research'}}
---
{'research': {'output': '### 概要\n\n**TravelPlanner** は、言語エージェントが実世界のシナリオで計画能力を評価するために設計された包括的なベンチマークです。旅行計画をテスト環境として利用し、ユーザーのニーズや常識的な制約を考慮しながら、言語エージェントがさまざまな検索ツールを使用して情報を収集し、計画を立てる能力を評価します。このベンチマークは、1,225のクエリを含み、クエリごとに複数の評価スクリプトを用いて、言語エージェントがユーザーのニーズと常識の制約の両方に合致する計画を作成できるかを判断します。\n\n### 新規性\n\nTravelPlannerは、実世界の複雑な計画をテーマにした言語エージェントの能力を評価するためのベンチマークであり、その新規性は以下の点にあります：\n- 実世界の旅行計画という具体的なシナリオを採用し、言語エージェントが多様な情報収集ツールを使用し、複数の制約条件（環境制約、常識的制約、厳格な制約）の下で合理的な計画を立てる能力を評価します。\n- クエリごとに一意の正解が存在しないことから、複数の評価スクリプトを用いて、計画の質を多角的に評価します。\n\n### 数式やアルゴリズムなどの手法\n\n具体的な数式やアルゴリズムの詳細は記載されていませんが、TravelPlannerは言語エージェントが以下のプロセスを経て計画を立てる能力を評価します：\n1. クエリを受け取り、ユーザーのニーズと制約条件を理解する。\n2. 様々な検索ツールを駆使して必要な情報を収集する。\n3. 収集した情報を基に、交通手段、日々の食事、観光地、宿泊施設を含む総合的な計画を立てる。\n\n### 実験結果\n\n- 大規模言語モデル（LLM）と計画戦略を用いた実験では、GPT-4-Turboが最も良い結果を示しました。\n- ツール使用エラーの分布や制約条件のクリア率など、様々な指標を用いてエージェントの計画能力を評価。\n- GPT-4-Turboを使用したケーススタディでは、ReActやDirect Planning、Reflexion Planningなどの戦略を用いたシナリオが示され、それぞれの戦略における

In [31]:
# DuckDuckago検索版
print(latest_output)

### 概要

**TravelPlanner** は、言語エージェントが実世界のシナリオで計画能力を評価するために設計された包括的なベンチマークです。旅行計画をテスト環境として利用し、ユーザーのニーズや常識的な制約を考慮しながら、言語エージェントがさまざまな検索ツールを使用して情報を収集し、計画を立てる能力を評価します。このベンチマークは、1,225のクエリを含み、クエリごとに複数の評価スクリプトを用いて、言語エージェントがユーザーのニーズと常識の制約の両方に合致する計画を作成できるかを判断します。

### 新規性

TravelPlannerは、実世界の複雑な計画をテーマにした言語エージェントの能力を評価するためのベンチマークであり、その新規性は以下の点にあります：
- 実世界の旅行計画という具体的なシナリオを採用し、言語エージェントが多様な情報収集ツールを使用し、複数の制約条件（環境制約、常識的制約、厳格な制約）の下で合理的な計画を立てる能力を評価します。
- クエリごとに一意の正解が存在しないことから、複数の評価スクリプトを用いて、計画の質を多角的に評価します。

### 数式やアルゴリズムなどの手法

具体的な数式やアルゴリズムの詳細は記載されていませんが、TravelPlannerは言語エージェントが以下のプロセスを経て計画を立てる能力を評価します：
1. クエリを受け取り、ユーザーのニーズと制約条件を理解する。
2. 様々な検索ツールを駆使して必要な情報を収集する。
3. 収集した情報を基に、交通手段、日々の食事、観光地、宿泊施設を含む総合的な計画を立てる。

### 実験結果

- 大規模言語モデル（LLM）と計画戦略を用いた実験では、GPT-4-Turboが最も良い結果を示しました。
- ツール使用エラーの分布や制約条件のクリア率など、様々な指標を用いてエージェントの計画能力を評価。
- GPT-4-Turboを使用したケーススタディでは、ReActやDirect Planning、Reflexion Planningなどの戦略を用いたシナリオが示され、それぞれの戦略におけるエージェントの振る舞いや計画の質が評価されています。

### 実装方法

TravelPlannerの実装について、具体的なコードやアルゴリズムの詳細は記載されていませんが

In [16]:
# Tavily検索版
print(latest_output)

「TravelPlanner: A Benchmark for Real-World Planning with Language Agents」というプロジェクトについて、以下の観点から詳しく説明します。

### 概要

TravelPlannerは、実世界のシナリオにおける言語エージェントの計画能力を複数の次元で評価するために設計された包括的なベンチマークです。具体的には、旅行計画をテスト環境として採用し、すべての関連情報を慎重に作成してデータの汚染を最小限に抑えています。このベンチマークは、各クエリに対して単一の正解を持たず、代わりに複数の事前定義された評価スクリプトを用いてテストされた計画を評価し、言語エージェントがツールを効果的に使用してクエリで指定された暗黙の常識と明示的なユーザーのニーズの両方に沿った計画を作成できるかどうかを判断します。

### 新規性

TravelPlannerの新規性は、言語エージェントが実世界の複雑な計画を行う能力を評価するために、旅行計画という具体的なシナリオを使用する点にあります。このベンチマークは、旅行計画の日数や厳格な制約の数を通じて、計画の幅と深さの両方でエージェントの能力を試験します。また、環境制約、常識制約、および厳格な制約という3種類の制約を設計することで、実世界のアプリケーションの観点から評価を行います。

### 手法

TravelPlannerは、交通手段、毎日の食事、アトラクション、そして宿泊施設を含む包括的な計画を立てるために、言語エージェントが様々なツールを使用して情報を収集し、意思決定を行いながら制約を満たすかどうかを評価します。具体的には、1,225のクエリが含まれ、訓練セット、検証セット、テストセットに分けられています。各クエリは、厳格な人間による検証を経て、実行可能な解決策が存在することが保証されています。

### 実験結果

既存の大規模言語モデル（LLMs）と計画戦略に関する実験結果は、TravelPlannerの検証セットとテストセットにおける異なるLLMsの性能を示しています。ツール使用エラーの分布や、GPT-4-Turboによる制約通過率、さらにはエージェントと参照との間の異なるツール使用回数の比較などが含まれます。

### 実装方法

TravelPlannerのデータ

In [15]:
stop

NameError: name 'stop' is not defined

In [ ]:
# query = """以下の論文について、概要、新規性、数式やアルゴリズムなどの手法、実験結果、実装方法等を順番に考えながら詳しく日本語で書きなさい。AI文章チェッカーによる判定で、AIが書いたと判定されないように書くこと。
# https://arxiv.org/pdf/2401.15884.pdf
# """
query = """2024/02/05から/06までの東京の天気を順番に詳しく教えてください。また、その気象になる理由を詳しく日本語で書き教えてください。"""
with Timer(prefix=f'Search agent operating time: '):
    latest_output = ""
    cnt = 0
    for s in blog_writer.stream(query, {"recursion_limit": 100}):
        if cnt == 4:
            break
        if "__end__" not in s:
            print(s)
            print("---")
            writing_output = (
                    s.get(RESEARCH_NODE, {}).get("output") or
                    s.get(WRITER_NODE, {}).get("output")
            )
            if writing_output:
                latest_output = writing_output
        cnt += 1

In [ ]:
print(latest_output)

In [ ]:
# 直接的にリンクを設定しないと無限ループに陥る可能性があります

In [ ]:
def search_agent(query):
    latest_output = ""
    for s in blog_writer.stream(query, {"recursion_limit": 100}):
        if "__end__" not in s:
            print(s)
            print("---")
            writing_output = (
                    s.get(RESEARCH_NODE, {}).get("output") or
                    s.get(WRITER_NODE, {}).get("output")
            )
            if writing_output:
                latest_output = writing_output
    return latest_output

In [ ]:
query = """以下の企業について、行っている事業と分野、これからやりそうなことや現在やっていいそうな事業を日本語で書きなさい。AI文章チェッカーによる判定で、AIが書いたと判定されないように書くこと。
https://www.ogis-ri.co.jp/
"""

In [ ]:
with Timer(prefix=f'Search agent operating time: '):
    search_output = search_agent(query)

In [ ]:
print(search_output)